In [37]:
import pandas as pd
import numpy as np

import math

If we first ignore reducing, then we have $\frac{1}{3} < \frac{n}{d} < \frac{1}{2} \implies \frac{d}{3} < n < \frac{d}{2} \implies \lceil \frac{d+1}{3} \rceil \leq n \leq \lfloor \frac{d-1}{2} \rfloor$. Also, we see that $d = 1,2,3,4$ will not produce any reduced fractions between $\frac{1}{3}$ and $\frac{1}{2}$. So we can start from $d = 5$.

Without optimizations, since it's only $12,\!000$ denominators to go through, it should be practical to just go through each one where $n$ is in the proper range, and just check if we have seen that fraction before.

In [72]:
def sieve(n):
    arr = [0,0,1] + [1,0]*(n//2 + 1)
    i = 3
    while i*i <= n:
        if arr[i]:  
            arr[i*i::2*i] = [0]*len(arr[i*i::2*i])
        i += 2

    ret = []
    for (i, p) in enumerate(arr):
        if p:
            ret.append(i)

    return arr, ret

In [73]:
pbs, ps = sieve(10**5)
gcd_cache = np.zeros((10**5, 10**5), int)
def gcd(a,b):
    if a == 0:
        return b

    if gcd_cache[a,b] == 0:
        gcd_cache[a,b] = math.gcd(b % a, a)
        gcd_cache[b,a] = gcd_cache[a,b]
    
    return gcd_cache[a,b]

In [75]:
cnt = 0
seen = set()
for d in range(5, 12000+1):
    for n in range(math.ceil((d+1)/3), math.floor((d-1)/2)+1):
        if pbs[d]:
            num, denom = n, d
        else:
            g = gcd(n,d)
            num, denom =  n // g, d // g

        if (num,denom) not in seen:
            seen.add((num,denom))
            cnt += 1

print(cnt)

7295372


We can start optimizing by finding mediants instead (i.e., using concepts from [Farey sequences][1]) and stopping once the denominator gets too big. Recursion is possible but it has risk of too much recursion depth, especially for even bigger Farey sequences.

So we go with iteration. Basically we start at the fraction right after $\frac{1}{3}$, and then loop through the Farey sequence until we reach $\frac{1}{2}$. Note, this works well because we only need to track the denominator and when it get to $2$, we are done (since if the denominator is $2$, then the numerator must be $1$). So, the only thing to work out is what would be the denominator of the next fraction.

From the section on [next terms of a Farey sequence][2], if the maximum allowable denominator is $D$, and the fraction I just passed has denominator $y$ (initialize $y=3$), and the fraction I am currently on has denominator $x$, then the next fraction in the sequence has denominator, $z$,

$$
z = x \cdot \left\lfloor \frac{D + y}{x} \right\rfloor - y = (D + y - [(D + y) \!\!\!\!\mod x]) - y = D - [(D + y) \!\!\!\!\mod x].
$$

For this problem, $D = 12000$, this suffices. However, if $D$ gets even larger, we need an even fast algorithm.

  [1]: https://en.wikipedia.org/wiki/Farey_sequence
  [2]: https://en.wikipedia.org/wiki/Farey_sequence#Next_term

In [121]:
def iterative_mediants(left_d, right_d, max_d):
    # denominator of initial mediant
    med_d = left_d + right_d

    # start with current denominator for fraction as close to left_d as possible
    while med_d + left_d <= max_d:
        med_d += left_d
    
    # prev_d initialized to left_d
    prev_d = left_d
    
    cnt = 0
    while med_d != right_d:
        # if prev_d, med_d are denominators next to each other in the Farey sequence,
        # the next fraction has denominator 
        # (med_d*(max_d + prev_d) // med_d) - prev_d = max_d - ((max_d + prev_d) % med_d)
        next_d = max_d - ((max_d + prev_d) % med_d)

        # shift one to the right
        prev_d = med_d
        med_d = next_d

        cnt += 1



    return cnt

iterative_mediants(3,2,100000)

506608484

The fraction rank algorithm described in [this talk][1] is much much faster for this! It basically produces all fractions of a given denominator, and uses a sort of sieve to reduce fractions.

  [1]: https://en.wikipedia.org/wiki/Farey_sequence

In [126]:
def fraction_rank(n, d, max_d):
    ret = np.zeros((max_d + 1), int)
    for i in range(max_d + 1):
        ret[i] = math.floor(i*n/d)
    
    for i in range(1,max_d + 1):
        ret[2*i::i] -= np.ones((len(ret[2*i::i])), int)*ret[i]

    return sum(ret)

fraction_rank(1, 2, 100000) - fraction_rank(1, 3, 100000) - 1

506608484